In [6]:
import os 
from openai import OpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader , TextLoader , PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.retrievers.multi_query import MultiQueryRetriever
import gradio as gr

In [2]:
AI_token='sk-vhUiTQZMrL3F5kg8VZKhqMypsciz3Zdq5Xr39igX8miy2lN5'

In [43]:
embedings=OpenAIEmbeddings(base_url='https://api.gapgpt.app/v1', api_key=AI_token)
def chunk_genarator(pdf,flag):
    
    db_name = "vector_db"
    loader=PyPDFLoader(pdf.name)
    document=loader.load()
    text_spliter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=150)
    chunks=text_spliter.split_documents(documents=document)
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name,embedding_function=embedings).delete_collection()
    vectorestore=Chroma.from_documents(documents=chunks,embedding=embedings,persist_directory=db_name)
    return vectorestore , gr.update(value='db created!')

In [5]:
MODEL = "gpt-4o-mini"
def Chat(message,history):
    
    llm=ChatOpenAI(base_url='https://api.gapgpt.app/v1', api_key=AI_token,
                  temperature=.7,model=MODEL)
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    retriver=MultiQueryRetriever.from_llm(llm=llm,
                                         retriever=vectorestore.as_retriver())
    conversation_chain=ConversationalRetrievalChain.from_llm(retriever=retriver,llm=llm,memory=memory)
    result = conversation_chain.invoke({"question": message})
    return result['answer']

In [60]:
with gr.Blocks(theme=gr.themes.Citrus()) as ui:
    gr.Markdown("<h1 style='text-align: center;'>🤖 RAG Chatbot 🤖</h1>")
    vector_store_state = gr.State(None)
    with gr.Row():
        file=gr.File(label='Upload your pdf file',file_types=['.pdf'],file_count='multiple',height=300)
        output=gr.Chatbot(height=300, type="messages")
    with gr.Row():
        create=gr.Button('create db',variant="primary")
        entry=gr.Textbox(label="Chat with our AI Assistant:")
        

    create.click(
        fn=chunk_genarator,
        inputs=[file],
        outputs=[vector_store_state, status_output] 
    )
    entry.submit(Chat, inputs=[entry, vector_store_state], outputs=[entry, output])
    
ui.launch()

C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1074: UserWarning: Expected 2 arguments for function <function chunk_genarator at 0x0000027F0F6AD940>, received 1.
  warnings.warn(
C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1078: UserWarning: Expected at least 2 arguments for function <function chunk_genarator at 0x0000027F0F6AD940>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7892
* To create a public link, set `share=True` in `launch()`.


C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:1031: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ParsRayaneh\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.

In [75]:
import gradio as gr
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
AI_token='sk-vhUiTQZMrL3F5kg8VZKhqMypsciz3Zdq5Xr39igX8miy2lN5' 
MODEL = "gpt-4o-mini"

embeddings = OpenAIEmbeddings(base_url='https://api.gapgpt.app/v1', api_key=AI_token)

def create_vector_db(pdf_files_list):
    
    if not pdf_files_list:
        return None, gr.update(value="upload pdf file")

    all_chunks = []
    try:
        for pdf_file in pdf_files_list:
            loader = PyPDFLoader(pdf_file.name)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
            chunks = text_splitter.split_documents(documents=documents)
            all_chunks.extend(chunks)


        vector_store = Chroma.from_documents(documents=all_chunks, embedding=embeddings)
        
        return vector_store, gr.update(value="✅")

    except Exception as e:
        return None, gr.update(value=f"error: {e}")


def chat_response(message, chat_history, vector_store):
    if vector_store is None:
        chat_history.append((message, "First upload pdf file"))
        return "", chat_history

    llm = ChatOpenAI(base_url='https://api.gapgpt.app/v1', api_key=AI_token, temperature=0.7, model=MODEL)

    retriver=MultiQueryRetriever.from_llm(llm=llm,
                                         retriever=vector_store.as_retriever())

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriver,
        memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    )

    result = conversation_chain.invoke({"question": message})

    chat_history.append((message, result['answer']))

    return "", chat_history


with gr.Blocks(theme=gr.themes.Citrus()) as ui:
    gr.Markdown("<h1 style='text-align: center;'>🤖 RAG Chatbot 🤖</h1>")
    
  
    vector_store_state = gr.State(None)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(
                label='Upload your pdf file',
                file_types=['.pdf'],
                file_count='multiple',
                height=300
            )
            create_button = gr.Button('Build vector', variant="primary")
            status_output = gr.Textbox(label="status", interactive=False)

        with gr.Column(scale=2):
            chatbot_output = gr.Chatbot(height=350)
            entry_input = gr.Textbox(label="Press enter")


    create_button.click(
        fn=create_vector_db,
        inputs=[file_input],
        outputs=[vector_store_state, status_output]
    )

  
    entry_input.submit(
        fn=chat_response,
        inputs=[entry_input, chatbot_output, vector_store_state],
        outputs=[entry_input, chatbot_output]
    )

ui.launch()

C:\Users\ParsRayaneh\AppData\Local\Temp\ipykernel_12700\168385434.py:78: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_output = gr.Chatbot(height=350)


* Running on local URL:  http://127.0.0.1:7907
* To create a public link, set `share=True` in `launch()`.


In [69]:
import gradio as gr
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# --- تنظیمات اولیه ---
# کلید API خود را اینجا قرار دهید
# بهتر است از متغیرهای محیطی استفاده کنید
AI_token='sk-vhUiTQZMrL3F5kg8VZKhqMypsciz3Zdq5Xr39igX8miy2lN5' 
MODEL = "gpt-4o-mini"

# یک بار مدل امبدینگ را تعریف کنید تا در هر فراخوانی دوباره ساخته نشود
embeddings = OpenAIEmbeddings(base_url='https://api.gapgpt.app/v1', api_key=AI_token)

# --- توابع اصلی برنامه ---

def create_vector_db(pdf_files_list):
    """
    لیستی از فایل‌های PDF را گرفته، آنها را پردازش کرده و یک وکتور استور Chroma می‌سازد.
    """
    if not pdf_files_list:
        return None, gr.update(value="⚠️ لطفاً حداقل یک فایل PDF آپلود کنید.")

    all_chunks = []
    try:
        for pdf_file in pdf_files_list:
            loader = PyPDFLoader(pdf_file.name)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
            chunks = text_splitter.split_documents(documents=documents)
            all_chunks.extend(chunks)

        # ساخت وکتور استور از تمام قطعات متنی
        vector_store = Chroma.from_documents(documents=all_chunks, embedding=embeddings)
        
        return vector_store, gr.update(value="✅ پایگاه دانش با موفقیت ساخته شد!")

    except Exception as e:
        return None, gr.update(value=f"❌ خطا در ساخت پایگاه دانش: {e}")


def chat_response(message, chat_history, vector_store):
    """
    با استفاده از وکتور استور و تاریخچه چت، به سوال کاربر پاسخ می‌دهد.
    """
    if vector_store is None:
        chat_history.append((message, "⚠️ لطفاً ابتدا یک فایل PDF آپلود و پایگاه دانش را ایجاد کنید."))
        return "", chat_history

    llm = ChatOpenAI(base_url='https://api.gapgpt.app/v1', api_key=AI_token, temperature=0.7, model=MODEL)
    
    # ساخت retriever از وکتور استور
    retriever = vector_store.as_retriever()

    # ساخت زنجیره مکالمه
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    )

    # فراخوانی زنجیره با سوال کاربر
    result = conversation_chain.invoke({"question": message})
    
    # اضافه کردن پرسش و پاسخ به تاریخچه
    chat_history.append((message, result['answer']))
    
    # ورودی کاربر را خالی کرده و تاریخچه جدید را برمی‌گردانیم
    return "", chat_history


# --- ساخت رابط کاربری با gr.Blocks ---

with gr.Blocks(theme=gr.themes.Soft()) as ui:
    gr.Markdown("<h1 style='text-align: center;'>🤖 RAG Chatbot 🤖</h1>")
    
    # کامپوننت نامرئی برای نگهداری وکتور استور
    vector_store_state = gr.State(None)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(
                label='فایل(های) PDF خود را آپلود کنید',
                file_types=['.pdf'],
                file_count='multiple',
                height=300
            )
            create_button = gr.Button('ساخت پایگاه دانش', variant="primary")
            status_output = gr.Textbox(label="وضعیت", interactive=False) # این کامپوننت تعریف شد

        with gr.Column(scale=2):
            chatbot_output = gr.Chatbot(height=350)
            entry_input = gr.Textbox(label="سوال خود را بپرسید و Enter را بزنید...")

    # --- تعریف رویدادها ---

    # 1. کلیک روی دکمه "ساخت پایگاه دانش"
    create_button.click(
        fn=create_vector_db,
        inputs=[file_input],
        outputs=[vector_store_state, status_output]  # خروجی‌ها به state و کادر وضعیت می‌روند
    )

    # 2. ارسال پیام در کادر ورودی
    entry_input.submit(
        fn=chat_response,
        inputs=[entry_input, chatbot_output, vector_store_state], # ورودی‌ها: پیام جدید، تاریخچه قبلی، وکتور استور
        outputs=[entry_input, chatbot_output] # خروجی‌ها: خالی کردن کادر ورودی، به‌روزرسانی چت‌بات
    )

ui.launch()

C:\Users\ParsRayaneh\AppData\Local\Temp\ipykernel_12700\3673928446.py:96: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_output = gr.Chatbot(height=350)


* Running on local URL:  http://127.0.0.1:7901
* To create a public link, set `share=True` in `launch()`.
